In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/데청캠 5조/Test_3

/content/drive/MyDrive/데청캠 5조/Test_3


In [ ]:
!pwd

/content/drive/MyDrive/데청캠 5조/Test_3


In [ ]:
import os
import cv2
import numpy as np
from moviepy.editor import VideoFileClip
import datetime
import time
from tqdm import notebook
import re
import glob

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2121728/45929032 bytes (4.6%)5636096/45929032 bytes (12.3%)8978432/45929032 bytes (19.5%)12320768/45929032 bytes (26.8%)15884288/45929032 bytes (34.6%)19308544/45929032 bytes (42.0%)22847488/45929032 bytes (49.7%)26312704/45929032 bytes (57.3%)29818880/45929032 bytes (64.9%)33210368/45929032 bytes (72.3%)36446208/45929032 bytes (79.4%)39632896/45929032 bytes (86.3%)

In [ ]:
!pip install pytube

     |████████████████████████████████| 55 kB 2.4 MB/s 


In [ ]:
from pytube import YouTube
from pytube import Playlist

In [ ]:
# Create_Folder 함수 : 폴더 생성 함수
def Create_Folder(Clean_File_name):
    # YouTube 객체에서 날짜 정보 불러오기
    if int(Clean_File_name) <= 3000:
      pl = Playlist("https://www.youtube.com/playlist?list=PLoMnIlrIuxWICw4rGjucBCASE_r-84yOO")
      yt = YouTube(pl.video_urls[int(Clean_File_name)])
    
    elif int(Clean_File_name) >= 10000 and int(Clean_File) <= 10800:
      pl = Playlist("https://www.youtube.com/playlist?list=PLoMnIlrIuxWJangLL014O-LPwRW8_wVWJ")
      yt = YouTube(pl.video_urls[int(Clean_File_name)])


    # 기준 시간 : 1년 1월 1일
    standard_time = datetime.datetime(1,1,1)

    # 결과 시간 = 파일 생성 시간 - 기준 시간
    result_time = int((yt.publish_date - standard_time).total_seconds())
    result_time = str(result_time)
    # print(result_time)


    # 000000.mp4 ~ 003000.mp4 : 뉴스데스크 영상 폴더
    if int(Clean_File_name) <= 3000:
      create_folder_dir = './main/NewsDesk/' + str(result_time)

    # 010000.mp4 ~ 010800.mp4 : 스포츠뉴스 영상 폴더
    elif int(Clean_File_name) >= 10000 and int(Clean_File) <= 10800:
      create_folder_dir = './main/Sports/' + str(result_time)

    try: 
        if not os.path.exists(create_folder_dir):
            os.makedirs(create_folder_dir)
            print(create_folder_dir, '폴더 생성')
            
    except OSError:
        print(create_folder_dir, ': 폴더 생성 오류')

    return create_folder_dir
  

In [ ]:
import moviepy.editor as mp
import re
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import os

In [ ]:
import ast
from moviepy.video.io.VideoFileClip import VideoFileClip

In [ ]:
def get_timeline(vid_name, script_2): 
    f = open(script_2, mode='rt', encoding='utf-8')
    sub2 = f.read()
    
    # 문자열 상태의 대본 -> type 자동 변환
    sub2 = ast.literal_eval(sub2)
    
    # '~니다, ~요, ~죠'가 들어있는 문장 번호 찾기 
    num_1 = next((i for i, item in enumerate(sub2) if '니다' in item['text']), None)
    num_2 = next((i for i, item in enumerate(sub2) if '요' in item['text']), None)
    num_3 = next((i for i, item in enumerate(sub2) if '죠' in item['text']), None)
    
    # 위에서 찾은 문장 번호가 리스트에 없으면 추가하기
    num_list = []
    if num_1 is not None:
        num_list.append(num_1)
    if num_2 is not None:
        num_list.append(num_2)
    if num_3 is not None:
        num_list.append(num_3)
    
    # 가장 작은 문장 번호 찾기
    number = min(num_list)
    # 영상 컷팅 끝나는 부분 지정하기
    end = sub2[number+1]['start']

    # 초반에 넘기는 부분 지정하기
    end3 = (item for item in sub2 if len(item['text']) > 2 and "음악" not in item['text'])
    dict3 = next(end3, False)
    start = dict3['start']
    
    return start, end

In [ ]:
#대본 만들기
def get_script(vid_name, script, folder_dir):
    f = open(script, mode='rt', encoding='utf-8')
    sub = f.read()
    
    sub = ast.literal_eval(sub)
    
    end = (item for item in sub if '니다' in item['text'])
    
    dict1 = next(end, False) #dict 1으로는 text 파일을 생성할 것
    num = next((i for i, item in enumerate(sub) if '니다' in item['text']), None)

    trans = []
    for i in range(num+1):
        trans.append(sub[i]['text'])
    
    script = ' '.join(trans)
    
    f = open(folder_dir + '/' +Clean_File_name + '.txt', 'w')
    f.write(script)
    f.close()
    
    return script

In [ ]:
def sound_extraction(Clean_File):
  MOV_File = './Preprocess/CUT_VID/' + Clean_File + '.mp4'
  clip = VideoFileClip(MOV_File)

  sound_file_name = './Preprocess/WAV/' + Clean_File + '.wav'
  clip.audio.write_audiofile(sound_file_name)

In [ ]:
def face_crop_vid (Clean_File_name):
  # 얼굴 인식 모델 가중치 파일
  face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')

  # [ 1단계 ] 얼굴 좌표 구하기 - 첫 프레임만 사용
  # 얼굴 좌표 구할 파일 정보
  File_name = './Preprocess/CUT_VID/' + Clean_File_name + '.mp4'
  print(File_name)
  video1 =  cv2.VideoCapture(File_name)

  width = video1.get(cv2.CAP_PROP_FRAME_WIDTH)
  height = video1.get(cv2.CAP_PROP_FRAME_HEIGHT)

  # 얼굴 좌표 구하기 작업
  ret, pic = video1.read()
  pic[0]

  # 얼굴 여러 개 인식시키기
  gray = cv2.cvtColor(pic, cv2.COLOR_BGR2GRAY)
  faces = face_cascade.detectMultiScale(gray, 
                                        scaleFactor=1.3, 
                                        minNeighbors=5, minSize=(20,20))
  

  for face in faces:
    # 얼굴 위치 계산용으로 변수에 임시 지정
    x_temp, y_temp, w_temp, h_temp = face 

    # 왼쪽 자리에 앵커 얼굴 있을 경우
    if x_temp < round(width*1/6 + 30):
      x, y, w, h = face
      print('type1 \n')
      break

    # 중앙 자리에 앵커 얼굴 있을 경우
    elif x_temp > round(width*2/6) and x_temp < round(width*3/6):
      x, y, w, h = face
      print('type2 \n')
      break

    # 오른쪽 자리에 앵커 얼굴 있을 경우
    elif x_temp > round(width*3/6) and x_temp < round(width*5/6):
      x, y, w, h = face
      print('type3 \n')
      break

  video1.release()


  # [2단계] 비디오 저장 처리 - 영상 속 모든 프레임 사용 
  video2 =  cv2.VideoCapture(File_name)
  
  # 저장할 파일 양식 지정
  fps = video2.get(cv2.CAP_PROP_FPS)
  fourcc = cv2.VideoWriter_fourcc(*'DIVX')
  width = video2.get(cv2.CAP_PROP_FRAME_WIDTH)
  height = video2.get(cv2.CAP_PROP_FRAME_HEIGHT)
  size = (160,160) # 해상도 160*160 지정

  out = cv2.VideoWriter('./Preprocess/FACE_VID/%s.avi' %Clean_File_name, fourcc, fps, size)
    
  # 파일 처리 작업
  while (video2.isOpened()):
    ret, frame = video2.read()

    if not ret:
        break

    cropped = frame[y - int(h/4):y + h + int(h/4), 
                    x - int(w/4):x + w + int(w/4)]
    
    cropped = cv2.resize(cropped, 
                         dsize=size, 
                         interpolation=cv2.INTER_AREA)

    # 위에서 자른 얼굴 위치를 토대로 프레임 저장 (연속적 프레임이므로 영상 형태)
    out.write(cropped)

  out.release()
  video2.release()


In [ ]:
from moviepy.editor import *

In [ ]:
def sound_synthesis(Clean_File):
  WAV_File = './Preprocess/WAV/' + Clean_File + '.wav'
  print(WAV_File)
  
  MOV_File = './Preprocess/FACE_VID/' + Clean_File + '.avi'
  print(MOV_File)
  
  videoclip = VideoFileClip(MOV_File)
  audioclip = AudioFileClip(WAV_File)
  new_audioclip = CompositeAudioClip([audioclip])
  
  videoclip.audio = new_audioclip
  videoclip.write_videofile('./Preprocess/FACE_VID/' + Clean_File + '.mp4')

In [ ]:
def Preprocess(Clean_File):
  # 폴더 생성하기
    Folder_dir = Create_Folder(Clean_File_name)

    # 원본 영상, 원본 대본 불러오기
    vid_name = './main/' + Clean_File_name + '.mp4'
    script = './main/' + Clean_File_name + '.txt'
    script_2 = './main/' + Clean_File_name + '_2.txt'

    # 편집 완료한 대본 파일 -> 'main/생성일_변환숫자' 폴더에 저장     
    trans = get_script(vid_name, script, Folder_dir) 
    
    start, end = get_timeline(vid_name, script_2)
    
    # 편집한 mp4 파일 -> 'Preprocess/CUT_VID' 폴더에 저장
    input_video_path = vid_name
    output_video_path = './Preprocess/CUT_VID/' + Clean_File_name + '.mp4' 
    
    with VideoFileClip(input_video_path) as video:
        new = video.subclip(start, end)
        new.write_videofile(output_video_path)

    # mp4 파일에서 음성 추출 -> [Output] wav 파일
    sound_extraction(Clean_File_name)

    # 얼굴 부위만 crop -> [Output] 소리 없는 얼굴 crop 영상
    face_crop_vid(Clean_File_name)

    # 소리 없는 얼굴 crop 영상에 음성 합성
    sound_synthesis(Clean_File_name)

    # fps 25로 영상 변환 -> 'main/생성일_변환숫자' 폴더에 저장     
    MOV_File = './Preprocess/FACE_VID/' + Clean_File_name + '.mp4'
    clip = VideoFileClip(MOV_File)
    New_File = Folder_dir + '/' + Clean_File_name + '.mp4'
    clip.write_videofile(New_File, fps=25, codec="mpeg4")
    

In [ ]:
# MOV_List : 영상 데이터 저장하는 변수
MOV_List = np.array(glob.glob('./main/*.mp4'))
print('MOV 파일 개수 :', len(MOV_List))

# TXT_List : 텍스트 데이터 저장하는 변수
TXT_List = np.array(glob.glob('./main/*.txt'))
print('TXT 파일 개수 :', len(TXT_List))

MOV 파일 개수 : 12
TXT 파일 개수 : 24


In [ ]:
Clean_File_List = np.array([])

for File in MOV_List:
    File_name = re.sub(pattern='./main/', repl='', string=File)
    File_name = re.sub(pattern='.mp4', repl='', string=File_name)
    Clean_File_List = np.append(Clean_File_List, [File_name])

print(Clean_File_List)
print('소속 위치, 확장자 뗀 본연의 파일명 개수 : ', len(Clean_File_List))

['000001' '000003' '000004' '000005' '000006' '000007' '000008' '000078'
 '000002' '000106' '002275' '001674']
소속 위치, 확장자 뗀 본연의 파일명 개수 :  12


In [ ]:
for Clean_File_name in notebook.tqdm(Clean_File_List):
  Preprocess(Clean_File_name)

  0%|          | 0/12 [00:00<?, ?it/s]

[MoviePy] >>>> Building video ./Preprocess/CUT_VID/000001.mp4
[MoviePy] Writing audio in 000001TEMP_MPY_wvf_snd.mp3



100%|██████████| 138/138 [00:00<00:00, 559.91it/s]

[MoviePy] Done.
[MoviePy] Writing video ./Preprocess/CUT_VID/000001.mp4




100%|██████████| 184/184 [00:06<00:00, 29.63it/s]


...

[MoviePy] Done.
[MoviePy] >>>> Video ready: ./Preprocess/FACE_VID/001674.mp4 

[MoviePy] >>>> Building video ./main/NewsDesk/63757843200/001674.mp4
[MoviePy] Writing audio in 001674TEMP_MPY_wvf_snd.mp3



100%|██████████| 230/230 [00:00<00:00, 514.58it/s]

[MoviePy] Done.
[MoviePy] Writing video ./main/NewsDesk/63757843200/001674.mp4




100%|█████████▉| 260/261 [00:00<00:00, 732.71it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: ./main/NewsDesk/63757843200/001674.mp4 

